# 序列化

我们把变量从内存中变成可存储或传输的过程称之为序列化，在Python中叫pickling，在其他语言中也被称之为serialization，marshalling，flattening等等，都是一个意思。

序列化之后，就可以把序列化后的内容写入磁盘，或者通过网络传输到别的机器上

反过来，把变量内容从序列化的对象重新读到内存里称之为反序列化，即unpickling

In [1]:
import pickle

d = dict(name='Bob', age=20, score = 88)
pickle.dumps(d)

b'\x80\x03}q\x00(X\x04\x00\x00\x00nameq\x01X\x03\x00\x00\x00Bobq\x02X\x03\x00\x00\x00ageq\x03K\x14X\x05\x00\x00\x00scoreq\x04KXu.'

pickle.dumps()方法把任意对象序列化成一个`bytes`，然后，就可以把这个`bytes`写入文件。或者用另一个方法`pickle.dump()`直接把对象序列化后写入一个`file-like Object`：

In [3]:
f = open('/tmp/dump', 'wb')
pickle.dump(d, f)
f.close()

当我们要把对象从磁盘读到内存时，可以先把内容读到一个bytes，然后用pickle.loads()方法反序列化出对象，也可以直接用pickle.load()方法从一个file-like Object中直接反序列化出对象。我们打开另一个Python命令行来反序列化刚才保存的对象：

In [10]:
f  = open('/tmp/dump', 'rb')
data = f.read()
d = pickle.loads(data)
print(d)
f.close()

{'name': 'Bob', 'age': 20, 'score': 88}


In [11]:
f  = open('/tmp/dump', 'rb')
d = pickle.load(f)
f.close()
print(d)

{'name': 'Bob', 'age': 20, 'score': 88}


Pickle的问题和所有其他编程语言特有的序列化问题一样，就是它只能用于Python，并且可能不同版本的Python彼此都不兼容

## Json

如果我们要在不同的编程语言之间传递对象，就必须把对象序列化为标准格式，比如`XML`，但更好的方法是序列化为`JSON`，因为`JSON`表示出来就是一个字符串，可以被所有语言读取，也可以方便地存储到磁盘或者通过网络传输。`JSON`不仅是标准格式，并且比`XML`更快，而且可以直接在Web页面中读取，非常方便。

JSON表示的对象就是标准的JavaScript语言的对象，JSON和Python内置的数据类型对应如下：

JSON类型|Python类型
---|---
{} | dict
[] | list
"string" | str
1234.56 | int/float
true/false | True/False
null | None

Python内置的json模块提供了非常完善的Python对象到JSON格式的转换。我们先看看如何把Python对象变成一个JSON：

In [13]:
import json

# 序列化
d = dict(name='Bob', age=20, score = 88)
print('json: ', json.dumps(d))

# 反序列化

json_str = '{"age": 20, "score": 88, "name": "Bob"}'
print('dict:', json.loads(json_str))

json:  {"name": "Bob", "age": 20, "score": 88}
dict: {'age': 20, 'score': 88, 'name': 'Bob'}


与pickle模块类似，json的dump和load接口提供了直接和一个`file-like object`进行交互的方式。

## Json进阶

我们上面看到了，我们可以用一个`dict`对象序列化与反序列化，那我们能否对一下对自定义对象进行序列化与反序列化呢。

当我们要对一个对象用json.dumps进行序列化时，必须额外提供一个将这个对象从class转化为dict的函数。

反之如果使用json.loads来加载一个对象，则要提供一个dict2object的函数。

In [27]:
import json

class Student(object):
    def __init__(self, name, age, score):
        self.name = name
        self.age = age
        self.score = score
s = Student('Bob', 20, 88)
# 如果直接对s进行序列化则会报异常
#json.dumps(s)

def student2dict(s):
    return {
        'name':s.name,
        'age': s.age,
        'score': s.score
    }
json.dumps(s, default=student2dict)

# 也可以使用每个class内置的一个__class__的方法, 也有少数例外，比如定义了__slots__的class。
json.dumps(s, default=lambda obj: obj.__dict__)

'{"name": "Bob", "age": 20, "score": 88}'

In [29]:
def dict2student(d):
    return Student(d['name'], d['age'], d['score'])

json_str = '{"age": 20, "score": 88, "name": "Bob"}'
s = json.loads(json_str, object_hook = dict2student)
print(s)